In [1]:
import json
import re
import requests
import pandas as pd

In [2]:
def get_code_series(medicament_name):
    
    url = f"https://www.open-medicaments.fr/api/v1/medicaments?limit=100&query={medicament_name}"
    res = requests.get(url)
    code_series = pd.read_json(res.text)["codeCIS"]
    
    return code_series

In [3]:
def get_drug_infos(codeCIS):
    
    url = f'https://www.open-medicaments.fr/api/v1/medicaments/{codeCIS}'
    res = requests.get(url)
    json_data = json.loads(res.text)
    composition = json_data['compositions'][0]['substancesActives'][0]
    name = composition['denominationSubstance']
    dosing = composition['dosageSubstance'].split()[0]
    unit = composition['dosageSubstance'].split()[1]
    type_ = json_data['formePharmaceutique']
    capsules_nb = re.findall(r"\d+", json_data["presentations"][0]["libelle"])[-1]
    price = json_data['presentations'][0]['prix']
    
    return (codeCIS, name, dosing, unit, type_, price, capsules_nb)

In [4]:
def to_mg(row):
    dosage = row.Dosage
    if row.Unité == "g":
        dosage = int(dosage)*1000
    return dosage

def convert_dosing_to_mg(df):
    df["Dosage"] = df.apply(lambda row : to_mg(row), axis=1)
    df["Unité"] = "mg"
    return df

In [5]:
medicament_name = "paracetamol"

df_drug_infos = get_code_series(medicament_name)[0:10].apply(get_drug_infos).apply(pd.Series)
df_drug_infos.columns = ["codeCIS", "Nom", "Dosage", "Unité", "Type", "Prix", "Nb_gélules"]

In [6]:
convert_dosing_to_mg(df_drug_infos)

,codeCIS,Nom,Dosage,Unité,Type,Prix,Nb_gélules
0,67445776,PARACÉTAMOL,500,mg,gélule,2.1,16
1,62721886,PARACÉTAMOL,1000,mg,comprimé,2.1,8
2,62772966,PARACÉTAMOL,1000,mg,comprimé,2.1,8
3,67034675,PARACÉTAMOL,500,mg,gélule,2.1,16
4,61223352,PARACÉTAMOL,500,mg,gélule,2.1,16
5,65085849,PARACÉTAMOL,500,mg,comprimé effervescent(e),2.1,16
6,61754805,PARACÉTAMOL,1000,mg,comprimé effervescent(e),2.1,8
7,60777377,PARACÉTAMOL,500,mg,comprimé effervescent(e),2.1,16
8,62502166,PARACÉTAMOL,500,mg,comprimé sécable,2.1,16
9,68830091,PARACÉTAMOL,1000,mg,comprimé sécable,NaN,8
